# Question 3: III. How do offered salaries compare to the prevailing wage?
> 	Are there job sub-categories that tend to get over-paid or under-paid?
> 	Are there companies that tend to over-pay or under-pay?
> 	Will the answer change if I take standard of living into account?